# Hidden Inverse Circuits for Coherent Error Supression 

**Key reference:** https://arxiv.org/abs/2104.01119

**Idea:** Self adjoint gates such as CX have two equivalent decompositons into hardware native gates. Find symmetric circuits and use both decompostions to cancel coherent errors.

If we have a matrix $M$ satisfying $M=M^\dagger$ we say that $M$ is self adjoint.
We can decompose this matrix M in two ways. 

$$
\begin{equation}
M = ABC = C^\dagger  B^\dagger  A^\dagger = M^\dagger
\end{equation}
$$

Here $M$ represents the unitary (and self-adjoint) matrix associated with a two qubit gate.


![alt text](images/inverses_screenshot.png "Title")

![alt text](images/gadget_screenshot.png "Title")

**My github repository:** https://github.com/CQCL/hidden_inverse_exp

## Getting the Hidden Inverses

In [1]:
from hseires_decompositions import get_hidden_inverse_circuits
from pytket.circuit.display import render_circuit_jupyter
from pytket.utils import compare_unitaries
import numpy as np

In [2]:
cx_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
#cz_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, -1]])

c1, c2 = get_hidden_inverse_circuits(cx_matrix)

render_circuit_jupyter(c1)
render_circuit_jupyter(c2)
print("Equivalent unitaries?", compare_unitaries(c1.get_unitary(), c2.get_unitary()))

Equivalent unitaries? True


## Phase and Pauli Gadgets

In [3]:
from utils.circuit_builders import get_phase_gadget, get_pauli_gadget
from phase_gadget_pass import phase_gadget_hi_pass

In [4]:
phase_gadget_circ = get_phase_gadget(0.7, 2)
render_circuit_jupyter(phase_gadget_circ)

In [5]:
phase_gadget_hi_pass.apply(phase_gadget_circ)

True

In [6]:
render_circuit_jupyter(phase_gadget_circ)

In [7]:
from pauli_gadget_pass import single_pauli_gadget_hi_pass, general_pauli_gadget_hi_pass

pauli_xyzy_circ = get_pauli_gadget("XYZ", 0.65)
render_circuit_jupyter(pauli_xyzy_circ)
u1 = pauli_xyzy_circ.get_unitary()

In [8]:
single_pauli_gadget_hi_pass.apply(pauli_xyzy_circ)
u2 = pauli_xyzy_circ.get_unitary()

In [9]:
render_circuit_jupyter(pauli_xyzy_circ)
print("Is the pass unitary preserving?", compare_unitaries(u1, u2))

Is the pass unitary preserving? True


In [10]:
from pytket import Circuit

xyzz = get_pauli_gadget("XYZZ", 0.65, decompose=False)
zyxx = get_pauli_gadget("ZYXX", 0.8, decompose=False)
xxxx = get_pauli_gadget("XXXX", 0.2, decompose=False)

circ = Circuit(4)
circ.append(xyzz)
circ.append(zyxx)
circ.append(xxxx)

stored_circ = circ.copy()
render_circuit_jupyter(circ)
u_before = circ.get_unitary()

In [11]:
from pytket.passes import SequencePass, RemoveRedundancies
from pauli_gadget_pass import general_pauli_gadget_hi_pass
from hseires_decompositions import hseries_squash

# idea for a custom sequence
custom_seq = SequencePass([general_pauli_gadget_hi_pass, hseries_squash, RemoveRedundancies()])

In [12]:
custom_seq.apply(circ)

True

In [13]:
from pytket import OpType

render_circuit_jupyter(circ)
u_after = circ.get_unitary()
print("ZZPhase count =",circ.n_gates_of_type(OpType.ZZPhase))

ZZPhase count = 16


In [14]:
print("Unitary matches?", compare_unitaries(u_before, u_after))

Unitary matches? True


## Alternating CNOT Decomposition

In [15]:
from alternating_cnot_decomposition import hidden_inverse_alternating_cnots

circuit = (
        Circuit(4)
        .H(0)
        .CX(0, 1)
        .H(1)
        .CX(1, 2)
        .H(3)
        .CX(2, 3)
        .H(3)
        .CX(0, 1)
        .H(1)
        .CX(1, 2)
        .CX(0, 1)
        .H(0)
        .H(1)
    )

u_initial = circuit.get_unitary()

In [16]:
render_circuit_jupyter(circuit)

In [17]:
hidden_inverse_alternating_cnots.apply(circuit)
u_final = circuit.get_unitary()
render_circuit_jupyter(circuit)

In [18]:
print("Unitary preserved?", compare_unitaries(u_initial, u_final))

Unitary preserved? True


## Ideas and Future Directions

1. Run some phase/pauli gadgets on H-series emulator/real device. Find out if coherent errors are suppressed
2. Test out this method for QEC gadgets (Maybe on real hardware? Talk more to Ben and Natalie).
3. Test this out with classical `ToffoliBox` circuits?
4. Figure out how this might play nicely with `OptimisePhaseGadgets` and `PauliSimp`.
5. Benchmark performance of alternating CNOT decomposition post `FullPeepholeOptimise`
6. Clean up and generalise code. Integrate into TKET somehow?
